# Homes Analysis in New York City

## Data Preparation 

In [ ]:
import pandas as pd
import numpy as np
import plotly
import plotly.io as pio
import plotly.express as px
from plotly.graph_objs import Layout,Scatter
from plotly.figure_factory._county_choropleth import create_choropleth
from plotly import graph_objs as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
import folium
from folium.plugins import MarkerCluster
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [ ]:
data = pd.read_csv ('https://raw.githubusercontent.com/YJawata/Loan_Analysis_NYC/master/LuxuryLoanPortfolio.csv') 

In [ ]:
data.shape

In [ ]:
data.head()

Remove space(s) from columns and rename '10_yr_treasury_index_date_funded' to 'ten_yr_treasury_index_date_funded'

In [ ]:
data.columns = data.columns.str.replace(' ', '_')
data.rename(columns={'10_yr_treasury_index_date_funded': 'ten_yr_treasury_index_date_funded'}, inplace=True)

In [ ]:
#check data type

data.info()

In [ ]:
#Gross square and land square feet should be float or integer instead of string

cols = ["LAND_SQUARE_FEET","GROSS_SQUARE_FEET"] 
data = (data.drop(cols, axis=1).join(data[cols].apply(pd.to_numeric, errors='coerce')))

In [ ]:
data.info()

In [ ]:
data.shape

Check missing data.

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(data.isnull(),cmap='viridis')

Based on the above data, some missing values are:

In [ ]:
missingdata = data.isnull().sum()/len(data)*100

print(pd.DataFrame([missingdata[missingdata>0],pd.Series(data.isnull().sum()[data.isnull().sum()>0])], index=['percent missing','number of missing data']))

Check for duplicates

In [ ]:
print(sum(data.duplicated()))
data[data.duplicated(keep=False)].sort_values(['loan_id']).head(10)

## Data Analysis

Check for outliers for property_value as this analysis will consider this parameter.

In [ ]:
fig = go.Figure(data=go.Scatter(
    y = data.property_value,
    x = data.funded_date,
    mode='markers',
    marker=dict(
        size=3,
        color=np.random.randn(500), #set color equal to a variable
        colorscale='Viridis', # one of plotly colorscales
        showscale=False
    )
))

fig.show()

As seen above, there is an outlier. According to the record, this value was derived from a purchase for a plane. As this record is a one-off sale and the occurence is not expected (according to yearly data). Therefore, it will be removed. --Assumption

In [ ]:
data.drop(data[data.property_value > 50000000].index, inplace=True)

In [ ]:
data.reset_index(drop=True)
data.shape

Choose parameters that are going to be used for the analysis.

In [ ]:
#Add year column based on funded_date

data['funded_date'] = pd.to_datetime(data['funded_date'])
data['Year'] = data['funded_date'].dt.year

In [ ]:
dhomes = data.filter(['Year','funded_amount','duration_years','ten_yr_treasury_index_date_funded',
                     'interest_rate_percent','property_value','GROSS_SQUARE_FEET',"LAND_SQUARE_FEET",'TOTAL_UNITS'], axis=1)

How do parameters correlate to each other?

In [ ]:
# I need to use matplotlib as docker has Authentication issue with chart_studio
corr = dhomes.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

Based on the above correlation plot, there are multiple parameters that have high correlation values. For instance: Interest rate vs Year, Interest rate vs duration_years, Gross square feet vs Property value, etc. Additionally, it can be assumed that gross square feet is equal to land square feet as the correlation between those parameters is high. Therefore, missing data is filled with the values from land square feet when data is available. -- Assumption 

In [ ]:
data.loc[data['LAND_SQUARE_FEET'].isnull(),'LAND_SQUARE_FEET'] = data['GROSS_SQUARE_FEET']
data.loc[data['GROSS_SQUARE_FEET'].isnull(),'GROSS_SQUARE_FEET'] = data['LAND_SQUARE_FEET']

In [ ]:
#check the remaining missing values

missingdata = data.isnull().sum()/len(data)*100

print(pd.DataFrame([missingdata[missingdata>0],pd.Series(data.isnull().sum()[data.isnull().sum()>0])], index=['percent missing','number of missing data']))

In [ ]:
#histogram to understand correlation result and its distribution

y = data.interest_rate_percent
x = data.Year

fig = go.Figure()
fig.add_trace(go.Histogram2dContour(
        x = x,
        y = y,
        colorscale = 'Blues',
        reversescale = True,
        name="",
        xaxis = 'x',
        yaxis = 'y'
    ))
fig.add_trace(go.Histogram(
        y = y,
        xaxis = 'x2',
        name="",
        marker = dict(
            color = 'rgba(0,0,0,1)'
        )
    ))
fig.add_trace(go.Histogram(
        x = x,
        yaxis = 'y2',
        name="",
        marker = dict(
            color = 'rgba(0,0,0,1)'
        )
    ))

fig.update_layout(
    autosize = False,
    xaxis = dict(
        zeroline = False,
        domain = [0,0.85],
        showgrid = False
    ),
    yaxis = dict(
        zeroline = False,
        domain = [0,0.85],
        showgrid = False
    ),
    xaxis2 = dict(
        zeroline = False,
        domain = [0.85,1],
        showgrid = False
    ),
    yaxis2 = dict(
        zeroline = False,
        domain = [0.85,1],
        showgrid = False
    ),
    height = 600,
    width = 600,
    bargap = 0,
    hovermode = 'closest',
    showlegend = False
)

fig.show()

## Dash

In [ ]:
# Mapping Data

data['ZIP']=data['ZIP_CODE'].astype(str)

In [ ]:
# import files to get lat long for each point and county name

nyc_zipcodes = pd.read_csv("https://raw.githubusercontent.com/YJawata/Loan_Analysis_NYC/master/zip_codes_states.csv", delimiter=',', dtype=str)
nyc_brg = pd.read_csv("https://raw.githubusercontent.com/YJawata/Loan_Analysis_NYC/master/nyc_neighboor.csv", delimiter=',', dtype=str)

In [ ]:
nyc_zipcodes = nyc_zipcodes.rename({'zip': 'ZIP'}, axis=1)
nyc_zipcodes.head()

In [ ]:
data['LAT'] = np.nan
data['LON'] = np.nan
data['COUNTY'] = np.nan

In [ ]:
# fill all the new columns with additional information

data['LAT'] = data['ZIP'].map(nyc_zipcodes.set_index('ZIP')['lat'])
data['LON'] = data['ZIP'].map(nyc_zipcodes.set_index('ZIP')['lon'])
data['COUNTY'] = data['ZIP'].map(nyc_brg.set_index('ZIP')['NBH'])
data.shape

In [ ]:
#convert to float from object

data["LAT"] = pd.to_numeric(data.LAT, errors='coerce')
data["LON"] = pd.to_numeric(data.LON, errors='coerce')

In [ ]:
data.reset_index(drop=True)

In [ ]:
#calculate average property value based on ZIP code for choropleth

mapnyc = data[['ZIP','property_value']].sort_values(by = ["ZIP"]).reset_index(drop=True)
mapnyc.groupby(['ZIP']).mean().reset_index(drop=True)
mapnyc

In [ ]:
map_nyc_homes = folium.Map(location=[40.7465, -74.0014], zoom_start=12.5, tiles='cartodbpositron')
map_nyc_homes.choropleth(geo_data="https://raw.githubusercontent.com/YJawata/Loan_Analysis_NYC/master/nyc_zip_code_tabulation_areas_polygons.geojson",  
             data=mapnyc,
             columns=['ZIP', 'property_value'], 
             key_on='feature.properties.postalcode', 
             nan_fill_color=None,
             nan_fill_opacity=0,
             fill_color='OrRd', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Property Value')

mc = MarkerCluster()
for i in data.itertuples():
    location = [i.LAT,  i.LON]
    tooltip = "City: {}<br> Neighboorhood: {}<br> Zipcode:{}<br> Click for more".format(i.CITY,i.COUNTY,i.ZIP_CODE)
    folium.Marker(location, 
                  popup="""<i>Property Value: </i> <br> <b>${}</b> <br>
                  <i>Total units: </i><b><br>{}</b><br>""".format(round(i.property_value,2), round(i.TOTAL_UNITS,2)), 
                  tooltip=tooltip).add_to(mc)
 
map_nyc_homes.add_child(mc)
map_nyc_homes

In [ ]:
#save file for dash

map_nyc_homes.save(outfile='example_map.html')

In [ ]:
#Create a sunburst to help visualising the data

data['Luxury']='NYC'
data['Category'] = np.where(data['property_value'] >= 3600000, 'more than 3.6M',
                            np.where(data['property_value'] <= 2000000, 'less than 2M','2M to 3.6M'))
sunburst=data[['Category','purpose','BUILDING_CLASS_CATEGORY']]
sunburst['count'] = 1
sunburst = sunburst.rename({'count': 'TOTAL_UNITS'}, axis=1)

In [ ]:
fig1=px.sunburst(sunburst,path=['Category','purpose','BUILDING_CLASS_CATEGORY'],values='TOTAL_UNITS',
               color= 'TOTAL_UNITS', width= 800, 
               color_continuous_scale = "RDBU_r")
fig1.update_layout(uniformtext=dict(minsize=10, mode='hide'))

In [ ]:
#check counts for each year

data.groupby(['Year']).size()

In [ ]:
#Create a graphical representation 

fig2 = make_subplots(specs=[[{"secondary_y": True}]])

# data strucutre
df=pd.DataFrame(data)
dfg=df.groupby('Year').sum().reset_index()

fig2.add_trace(
    go.Bar(
        x=dfg.Year,
        y=dfg.funded_amount,
        orientation='v',
        marker_color="#33CFA5",
        name="Total Loan Amount in USD"),
    secondary_y=False,
)

fig2.add_trace(
    go.Bar(
        x=dfg.Year,
        y=dfg.GROSS_SQUARE_FEET,
        orientation='v',
        marker_color="RebeccaPurple",
        name="Total Gross Square Feet"),
    secondary_y=False,
)

fig2.add_trace(
    go.Box(
        x=data['Year'], 
        y=data['interest_rate_percent'],
        marker_color = '#FF851B',
        name="Interest Rate"),
    secondary_y=True,
)

fig2.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            buttons=list([
                dict(label="Loan Amount",
                     method="update",
                     args=[{"visible": [True, False, True]},
                           {"title": "Yearly Total Loan Amount vs Interest Rate",
                            "annotations": []}]),
                dict(label="Gross Feet Square",
                     method="update",
                     args=[{"visible": [False, True, True]},
                           {"title": "Yearly Total Gross Feet Square Funded vs Interest Rate",
                            "annotations": []}]),
            
            ]),
        )
    ])


# Add figure title
fig2.update_layout(hovermode='closest')
fig2.update_traces(marker_line_width=0.5, opacity=0.6)

# Set x-axis title
fig2.update_xaxes(title_text="Year")

# Set y-axes titles
fig2.update_yaxes(title_text="<b>Interest Rate</b>", range = [0,5], secondary_y=True)

fig2.show()

In [ ]:
# Gross square feet funded based on purpose
gsc = data[['Year','purpose','GROSS_SQUARE_FEET']].sort_values(by = ["Year"]).reset_index(drop=True)
gscchart = gsc.groupby(['Year', 'purpose'])['GROSS_SQUARE_FEET'].sum().reset_index()
gscchartfinal = gscchart.pivot(index='Year', columns='purpose', values='GROSS_SQUARE_FEET')
gscchartfinal.head()

In [ ]:
fig3 = go.Figure()


for col in gscchartfinal.columns:
    fig3.add_trace(go.Scatter(x=gscchartfinal.index, y=gscchartfinal[col], name = col))
    
# Set title
fig3.update_layout(title=dict(text='Yearly Total Gross Square Feet Funded Based on Purposes'),legend_orientation="h",hovermode = 'x')
fig3.update_xaxes(title_text="Year")
fig3.update_yaxes(title_text="Gross Square Feet")
fig3.show()

In [ ]:
#create dataframe for Total funded_amount per purpose
linehomes = data[['Year','purpose','funded_amount']].sort_values(by = ["Year"]).reset_index(drop=True)
linehomeschart = linehomes.groupby(['Year', 'purpose'])['funded_amount'].sum().reset_index()
linechartfinal = linehomeschart.pivot(index='Year', columns='purpose', values='funded_amount')
linechartfinal.head()

In [ ]:
fig4 = go.Figure()


for col in linechartfinal.columns:
    fig4.add_trace(go.Scatter(x=linechartfinal.index, y=linechartfinal[col], name = col))
    fig4.add_trace(
    go.Scatter(x=linechartfinal.index,
               y=[linechartfinal[col].mean()] * len(linechartfinal.index),
               name="Average Yearly Total Loan Amount",
               visible=False,
               line=dict(color="#F06A6A", dash="dash")))

# Buttons
fig4.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="All",
                     method="update",
                     args=[{"visible": [True, False,True, False, True,False, True, False, True, False]},
                           {"title": "All",
                            "annotations": []}]),
                dict(label="Boat",
                     method="update",
                     args=[{"visible": [True, True, False, False, False,False, False, False, False, False]},
                           {"title": "Total Loan Amount for Boat",
                            "annotations": []}]),
                dict(label="Commerical Property",
                     method="update",
                     args=[{"visible": [False, False, True, True, False, False, False, False, False, False]},
                           {"title": "Total Loan Amount for Commerical Property",
                            "annotations": []}]),
                dict(label="Home",
                     method="update",
                     args=[{"visible": [False, False, False, False, True,True, False, False, False, False]},
                           {"title": "Total Loan Amount for Home",
                            "annotations": []}]),
                dict(label="Investment Property",
                     method="update",
                     args=[{"visible": [False, False, False, False, False,False, True, True, False, False]},
                           {"title": "Total Loan Amount for Investment Property",
                            "annotations": []}]),
                dict(label="Plane",
                     method="update",
                     args=[{"visible": [False, False, False, False, False, False, False,False, True, True]},
                           {"title": "Total Loan Amount for Plane",
                            "annotations": []}]),
                
            ]),
        )
    ])

# Set title
fig4.update_layout(title=dict(text='Yearly Funded Amount on Based on Purposes'),hovermode = 'x')
fig4.update_xaxes(title_text="Year")
fig4.update_yaxes(title_text="Funded Amount")
fig4.show()

In [ ]:
# choose parameters that have high correlations

scatter = data[["GROSS_SQUARE_FEET","BUILDING_CLASS_CATEGORY","duration_years","interest_rate_percent","Year","property_value","funded_amount","purpose","COUNTY"]].sort_values(by=["Year"]).reset_index(drop=True)

In [ ]:
fig5 = px.scatter(scatter, x="funded_amount", y="interest_rate_percent", animation_frame="Year", 
                 animation_group="BUILDING_CLASS_CATEGORY", size="property_value", color="purpose", facet_col="purpose",
                 hover_name="BUILDING_CLASS_CATEGORY",log_x=False, size_max=45, range_x=[100000,20000000], range_y=[2,5])

fig5.update_layout(title=dict(text='Interest Rate vs Purpose vs Building Category'))

fig5.show()

In [ ]:
#initiate dash

app = dash.Dash()

app.layout = html.Div([
    html.Div([
        html.Div([
            html.H1("Loan Application Based on Purposes in NYC"),
            html.Iframe(id='map', srcDoc=open('example_map.html', 'r').read(), width='100%', height='800')
        ], className="six columns"),
        
        html.Div([
            html.H2( ),
            dcc.Graph(id='g1', figure=fig1,style={'width': '100%'
        })
        ], className="six columns"),
        
        html.Div([
            html.H3( ),
            dcc.Graph(id='g2', figure=fig2,style={'width': '100%'
        })
        ], className="six columns"),
        
        
        html.Div([
            html.H4( ),
            dcc.Graph(id='g4', figure=fig4,style={'width': '100%'
        })
        ], className="six columns"),
        

        html.Div([
            html.H5(),
            dcc.Graph(id='g5', figure=fig5,style={'width': '100%'
        })
        ], className="six columns"),
    ], className="row")
])

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

if __name__ == '__main__':
    app.run_server(host="localhost",port=8050,debug = False)
